In [20]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import re

In [21]:
# Read the raw data
json_file = pd.read_json('../data/raw/current_rent_info.json')
df = pd.DataFrame(json_file.values.tolist()).T
df = df.rename(columns={0: 'address', 1: 'cost', 2: 'rooms', 3: 'parking', 4: 'type', 5: 'num_schools', 6: 'desc'})
print(df.shape)
display(df.head())

(11990, 7)


,address,cost,rooms,parking,type,num_schools,desc
0,"1302/1 Powlett Street, East Melbourne VIC 3002",$1795 Per Week,"[3 Beds, 2 Baths]",[2 Parking],Apartment / Unit / Flat,12,"Perfectly positioned overlooking the MCG, Tenn..."
1,"213/150 Clarendon Street, East Melbourne VIC 3002","$2,200","[2 Beds, 2 Baths]",[1 Parking],Apartment / Unit / Flat,12,JORDAN PALMA
2,"109/150 Clarendon Street, East Melbourne VIC 3002",$1595 per week,"[2 Beds, 2 Baths]",[2 Parking],Apartment / Unit / Flat,12,Looking directly over magnificent Fitzroy Gard...
3,"15 Berry Street, East Melbourne VIC 3002","$2,600.00 Per Week","[4 Beds, 3 Baths]",[2 Parking],House,13,This amazing town residence is bathed througho...
4,East Melbourne VIC 3002,"$1,500 per week","[1 Bed, 1 Bath]",[1 Parking],Apartment / Unit / Flat,13,"class=""css-dxogle"">* Unverified feature<svg a..."


In [22]:
# Get the cost 
def get_cost(cost):
    
    # Capture weekly cost
    pattern = re.compile(r'\s*\$\s*(\d+(?:[.,]\d+)*)\s*(?:\/\s*\$?\d+(?:[.,]\d+)*)?\/?((?:\s*(p[.\/]?w\.?|weekly|.*week|W|wk|.*(?:fully\s*)?furnished|incl|\*{2}).*)|$)', re.IGNORECASE)
    # Weekly cost - without $
    pattern1 = re.compile(r'\s*(\d+(?:[.,]\d+)*)\s*(?:\/\s*\$?\d+(?:[.,]\d+)*)?\/?((?:\s*(p[.\/]?w\.?|weekly|.*week|W|wk|.*(?:fully\s*)?furnished|incl|\*{2}).*)|$)', re.IGNORECASE)
    # Capture monthly cost
    pattern2 = re.compile(r'\s*\$?\s*(\d+(?:[.,]\d+)*)\s*(?:\s*(.*month|p\.?(?:c)?\.?m\.?|monthly|mth)).*', re.IGNORECASE)
    # Capture annually cost
    pattern3 = re.compile(r'\s*\$?\s*(\d+(?:[.,]\d+)*)\s*(?:\s*(p\.?a\.?|per (year|annum)|annually)).*', re.IGNORECASE)

    match = pattern.search(cost)
    match1 = pattern1.search(cost)
    match2 = pattern2.search(cost)
    match3 = pattern3.search(cost)
    
    # Weekly
    if match:
        return float(match.group(1).replace(',', ''))
    elif match1:
        return float(match1.group(1).replace(',', ''))
    
    # Monthly
    elif match2:
        return float(match2.group(1).replace(',', '')) / 4
    
    # Annually
    elif match3:
        return float(match3.group(1).replace(',', '')) / 52
    else:
        return None

In [23]:
# Get the cost
df['cost_1'] = df['cost'].apply(get_cost)

In [24]:
# 1658, 10861, 0, 1 - Examples of results after regexing
df.iloc[[0,1,1658,10861]][['cost', 'cost_1']].rename(columns={'cost': 'before_regex', 'cost_1': 'after_regex'})

,before_regex,after_regex
0,$1795 Per Week,1795.0
1,"$2,200",2200.0
1658,$260 per month,65.0
10861,"$52,000 per annum plus GST",1000.0


In [25]:
# Incorrect cost (manually searching)
df.iloc[[2795, 11690]]

,address,cost,rooms,parking,type,num_schools,desc,cost_1
2795,"33 River St, Maribyrnong VIC 3032",AVAILABLE BY 1/9/2024,"[6 Beds, 3 Baths]",[3 Parking],House,10,The home's current floorplan comprises six gen...,9.0
11690,"40 Nautilus Street, Rye VIC 3941","$800 1,2 or 3 Month Lease","[4 Beds, 2 Baths]",[2 Parking],House,1,"class=""css-dxogle"">* Unverified feature<svg a...",200.0


In [26]:
# Remove wrong cost
ind = df[df['cost'] == 'AVAILABLE BY 1/9/2024'].index
df = df.drop(ind, axis=0)

# Change to the correct cost
df.loc[df['cost'] == '$800 1,2 or 3 Month Lease', 'cost_1'] = 800

In [27]:
# Get all the data with not NA cost
df = df.drop(columns=['cost'])
df = df.rename(columns={'cost_1': 'cost'})
df = df[df['cost'].notna()]

In [28]:
# Get the temporary suburb name (since there are some suburbs contain multiple smaller areas) from the address
def get_temp_suburb_name(address):
    pattern = re.compile(r'.*, (.*) VIC \d{4}')
    match = pattern.search(address)
    if match:
        return match.group(1)
    return None

In [29]:
df['suburb'] = df['address'].apply(get_temp_suburb_name)

# Remove address with wrong format: "address, suburb VIC postcode"
df = df[df['suburb'].notna()]

In [30]:
# Get number of beds and baths
def get_rooms(rooms):
    if len(rooms) > 0:
        beds = rooms[0].split()[0]
        baths = rooms[1].split()[0]
        beds = int(beds) if beds.isdigit() else 0
        baths = int(baths) if baths.isdigit() else 0
        return beds, baths
    return 0, 0
    
 # Get number of parking slots   
def get_parking(parking): 
    if len(parking) > 0:
        parkings = parking[0].split()[0]
        return int(parkings) if parkings.isdigit() else 0
    return 0

In [31]:
df[['beds', 'baths']] = df['rooms'].apply(lambda r: pd.Series(get_rooms(r)))
df['parking'] = df['parking'].apply(get_parking)

In [32]:
display(df['type'].value_counts())
sum((df['type'].value_counts()).iloc[:2])/sum(df['type'].value_counts())

type
House                            5476
Apartment / Unit / Flat          4755
Townhouse                        1134
Studio                            178
Villa                              19
Carspace                           17
New House & Land                   12
New Apartments / Off the Plan      10
Acreage / Semi-Rural                9
Semi-Detached                       7
Terrace                             7
Block of Units                      5
Duplex                              5
Vacant land                         1
New land                            1
Name: count, dtype: int64

0.8792540391887247

In [33]:
# Remove all except first 2 (sufficient data + match with historical data)
df = df[df['type'].isin((df['type'].value_counts()).index[:2])]

In [34]:
# Remove carpark, storage (based on address)
df = df.drop(df[df['address'].str.contains('carpark|car park|storage|cp|carspace|car space|parking|lot', case=False)]['address'].index, axis=0)

In [35]:
# Assume that a property has >= 1 bed and >= 1 bath
df = df.drop(df[(df['beds'] == 0) | (df['baths'] == 0)].index, axis=0)

In [36]:
# Drop rooms (because we have beds and baths)
df = df.drop(columns=['rooms'])

In [37]:
df.sort_values('cost').head(10)

,address,parking,type,num_schools,desc,cost,suburb,beds,baths
11721,"16 Watson Road, Leongatha VIC 3953",0,House,6,34 STORAGE UNITS IN LEONGATHA \nSizes range fr...,31.25,Leongatha,1,1
11837,"40-42 Watts Road, Nyora VIC 3987",0,Apartment / Unit / Flat,1,SHIPPING CONTAINER STORAGE UNIT AVAILABLE\nShi...,34.52,Nyora,1,1
11835,"40-42 Watts Road, Nyora VIC 3987",0,House,1,SHIPPING CONTAINER STORAGE UNIT AVAILABLE\nShi...,34.52,Nyora,1,1
4226,"68 Hayward Lane, Melbourne VIC 3000",1,House,12,Suitable for motorbikes or motorcycles only.,37.50,Melbourne,1,1
4186,"2512/22-24 Jane Bell Lane, Melbourne VIC 3000",1,Apartment / Unit / Flat,11,This listing is for car park only,60.00,Melbourne,1,1
4066,"2657/181 Exhibition Street, Melbourne VIC 3000",1,Apartment / Unit / Flat,12,Tired of looking for parking?,65.00,Melbourne,1,1
11313,"1C Dodemaides, Trafalgar VIC 3824",3,House,3,"class=""css-1sv4kwn"">Sorry, no image available.",70.00,Trafalgar,1,1
1652,"7P5/70 Southbank Boulevard, Southbank VIC 3006",1,Apartment / Unit / Flat,10,One secure car space available for renting in ...,80.00,Southbank,1,1
8987,"30 Daly Boulevard, Highton VIC 3216",2,House,7,"class=""css-dxogle"">* Unverified feature<svg a...",130.00,Highton,9,2
7401,"16 Waranga Street, Dandenong North VIC 3175",0,House,4,"class=""css-dxogle"">* Unverified feature<svg a...",150.00,Dandenong North,7,1


In [38]:
# Remove car space, parking (based on desc)
df = df.drop(df.sort_values('cost')[:8].index, axis=0)

In [39]:
# Remove unncessary features
df = df.drop(columns=['desc'])

In [40]:
# Drop duplicates
df = df.drop_duplicates()

In [41]:
# Ratio of cost/rooms
df['cost/(beds+baths)'] = df['cost']/(df['beds'] + df['baths'])

In [42]:
# low cost/(beds+baths)
display(df.sort_values('cost/(beds+baths)')[['cost', 'beds', 'baths', 'cost/(beds+baths)']][:3])
# high cost/(beds+baths)
display(df.sort_values('cost/(beds+baths)')[['cost', 'beds', 'baths', 'cost/(beds+baths)']][-3:])

,cost,beds,baths,cost/(beds+baths)
8987,130.0,9,2,11.818182
4865,160.0,8,2,16.000000
7395,220.0,7,5,18.333333


,cost,beds,baths,cost/(beds+baths)
184,5750.0,2,2,1437.500000
182,12500.0,4,4,1562.500000
600,95000.0,12,3,6333.333333


In [43]:
# Calculate 0.5th percentile (lower) and 99.5th percentile (upper) of the ratio
lower = df['cost/(beds+baths)'].quantile(0.005)
upper = df['cost/(beds+baths)'].quantile(0.995)

# Filtering all properties with lower <= ratio <= upper
df1 = df[(df['cost/(beds+baths)'] >= lower) & (df['cost/(beds+baths)'] <= upper)]

In [44]:
df1['address'].count()

np.int64(9926)

In [45]:
# Check some non unique address
non_unique_address = df1['address'].value_counts()[df1['address'].value_counts() > 1].index
df1[df1['address'].isin(non_unique_address)].sort_values('address').head(10)


,address,parking,type,num_schools,cost,suburb,beds,baths,cost/(beds+baths)
8823,"13 Dean Street, Belmont VIC 3216",1,House,6,440.0,Belmont,3,1,110.000000
8797,"13 Dean Street, Belmont VIC 3216",1,House,6,460.0,Belmont,3,1,115.000000
2146,"165/283 Spring Street, Melbourne VIC 3000",1,Apartment / Unit / Flat,14,650.0,Melbourne,2,2,162.500000
2050,"165/283 Spring Street, Melbourne VIC 3000",1,Apartment / Unit / Flat,14,660.0,Melbourne,2,2,165.000000
2128,"1B1B/134-160 Spencer Street, Melbourne VIC 3000",0,Apartment / Unit / Flat,9,650.0,Melbourne,1,1,325.000000
2873,"1B1B/134-160 Spencer Street, Melbourne VIC 3000",0,Apartment / Unit / Flat,9,580.0,Melbourne,1,1,290.000000
11917,"24 Viola Circuit, Clyde VIC 3978",1,House,3,550.0,Clyde,3,2,110.000000
11879,"24 Viola Circuit, Clyde VIC 3978",1,House,3,590.0,Clyde,3,2,118.000000
1910,"27 Otago Grove, Werribee VIC 3030",3,House,3,565.0,Werribee,4,2,94.166667
2048,"27 Otago Grove, Werribee VIC 3030",3,House,3,525.0,Werribee,4,2,87.500000


In [47]:
df1.head()

,address,parking,type,num_schools,cost,suburb,beds,baths,cost/(beds+baths)
0,"1302/1 Powlett Street, East Melbourne VIC 3002",2,Apartment / Unit / Flat,12,1795.0,East Melbourne,3,2,359.000000
2,"109/150 Clarendon Street, East Melbourne VIC 3002",2,Apartment / Unit / Flat,12,1595.0,East Melbourne,2,2,398.750000
3,"15 Berry Street, East Melbourne VIC 3002",2,House,13,2600.0,East Melbourne,4,3,371.428571
5,"8.15/280 Albert Street, East Melbourne VIC 3002",1,Apartment / Unit / Flat,13,1450.0,East Melbourne,2,2,362.500000
6,"21/322 Albert St, East Melbourne VIC 3002",1,Apartment / Unit / Flat,12,1100.0,East Melbourne,2,2,275.000000


In [48]:
df1.to_csv('../data/curated/preprocessed_current_rent.csv', index=False)